In [3]:
import pandas as pd
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing


In [17]:
house= fetch_california_housing()
house

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [18]:
#data preparation

data=pd.DataFrame(house.data,columns=house.feature_names)
data['Price']=house.target

In [25]:
X=data.iloc[:,:-1]
y= data.iloc[:,-1]

In [28]:
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [31]:
#train _Test split

x_train,x_test,y_train,y_test= train_test_split(X,y,test_size=.2)
signature = infer_signature(x_train,y_train)
param_grid={
 'n_estimators':[100,200],
 'max_depth':[5,10,None],
 'min_samples_split':[2,5],
 'min_samples_leaf': [1,2]
}

In [39]:
# hyperparameter tuning using grid search cv
def hyper_param(x_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,verbose=2,scoring='accuracy')
    grid_search.fit(x_train,y_train)
    return grid_search


In [41]:
# Start Mlflow Experiment
with mlflow.start_run():
    grid= hyper_param(x_train,y_train,param_grid)
    #get best model
    best_model= grid.best_estimator_
    #eval
    y_pred= best_model.predict(x_test)
    mse= mean_squared_error(y_test,y_pred)

    #log_ best param and metrics
    mlflow.log_param("best_n_estimators",grid.best_params_['n_estimators'])
    mlflow.log_param("best max_depth",grid.best_params_['max_depth'])
    mlflow.log_param("min_samples_split",grid.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf",grid.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)

    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url_type_store!='file':
        mlflow.sklearn.log_model(best_model,"model",registered_model_name="Best Rf")
    else:
        mlflow.sklearn.load_model(best_model,"model",signature=signature)
    
    print(f"Best hyperpara:{grid.best_params_}")
    print(f"mse:{mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\s

[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s


d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\s

[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.6s


...

d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "d:\Mlops_Udemy\mlflow\iris\venv\lib\s

[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=  12.4s


2025/03/16 17:30:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best Rf'.
2025/03/16 17:30:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Rf, version 1


Best hyperpara:{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
mse:0.430700445842394
🏃 View run delightful-fowl-506 at: http://127.0.0.1:5000/#/experiments/0/runs/63929eaa3e3d417cab6dfeba85458081
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best Rf'.
